In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
#import pandas_profiling

In [2]:
# ## plotly 
# import plotly.plotly as py
# import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode(connected=True)

In [3]:
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:50].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    #plt.savefig('lgbm_importances01.png')

In [4]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

## FE

In [5]:
# trace = go.Scatter(
#     x = np.linspace(0, 1, 100),
#     y = train_df.iloc[0,2:],
#     mode = 'lines+markers'
# )

# data = [trace]

# # Plot and embed in ipython notebook!
# iplot(data)

In [6]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [9]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190_rank,var_191_rank,var_192_rank,var_193_rank,var_194_rank,var_195_rank,var_196_rank,var_197_rank,var_198_rank,var_199_rank
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,0.600595,0.129952,0.786795,0.342780,0.560603,0.051698,0.828570,0.371510,0.152260,0.554508
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,0.825745,0.542247,0.669408,0.978745,0.229170,0.929432,0.841812,0.461605,0.780358,0.653895
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,0.475550,0.777698,0.439045,0.342330,0.855973,0.985463,0.050425,0.255628,0.353152,0.602302
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,0.603220,0.205107,0.214713,0.319368,0.948815,0.227970,0.200057,0.932102,0.740313,0.314173
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,0.159965,0.750988,0.081410,0.932045,0.065100,0.177920,0.589260,0.720267,0.743213,0.319582


In [21]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190_rank,var_191_rank,var_192_rank,var_193_rank,var_194_rank,var_195_rank,var_196_rank,var_197_rank,var_198_rank,var_199_rank
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,2.663879,0.515158,2.467704,0.579641,10.383872,-0.123960,6.527806,3.181426,1.945928,-0.605189
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,6.310426,4.186910,1.729548,10.718824,3.536208,1.890373,6.841158,4.057000,14.324242,1.276272
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,1.381806,7.614047,0.733381,0.577100,18.492601,3.096028,-0.328837,2.113400,5.199182,0.238813
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,2.694342,0.972886,0.154121,0.453949,21.855669,-0.289659,-0.585668,9.593385,13.303194,-2.827427
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-0.238428,7.150452,-0.012277,8.569408,0.865023,-0.269033,2.313847,6.844774,13.375893,-2.815650


In [7]:
var_list = [f for f in train_df.columns if 'var' in f]
rank_var_list = [f+'_rank' for f in train_df.columns if 'var' in f]
sort_var_list = ['sort_var_' + str(i) for i in range(len(var_list))]

In [20]:


# train_df['mean'] = np.mean(train_df[var_list],axis=1)
# train_df['std'] = np.std(train_df[var_list],axis=1)
# train_df['min'] = np.min(train_df[var_list],axis=1)
# train_df['max'] = np.max(train_df[var_list],axis=1)
# train_df['sum'] = np.sum(train_df[var_list],axis=1)
# train_df['median'] = np.median(train_df[var_list],axis=1)

# train_df[mean_var_list] = (train_df[var_list]-np.median(train_df[var_list],axis=0))/((np.max(train_df[var_list],axis=0)-np.median(train_df[var_list],axis=0)))
# test_df[mean_var_list] = (test_df[var_list]-np.median(test_df[var_list],axis=0))/((np.max(test_df[var_list],axis=0)-np.median(test_df[var_list],axis=0)))


train_df[rank_var_list] = train_df[var_list].rank(method='average',pct=True)
test_df[rank_var_list] = test_df[var_list].rank(method='average',pct=True)

train_df[rank_var_list] = train_df[rank_var_list].multiply(train_df[var_list].values)
test_df[rank_var_list]  = test_df[rank_var_list].multiply(test_df[var_list].values)



# #simple stats on rank cols 
# train_df['mean'] = np.mean(train_df[rank_var_list],axis=1)
# train_df['std'] = np.std(train_df[rank_var_list],axis=1)
# train_df['min'] = np.min(train_df[rank_var_list],axis=1)
# train_df['max'] = np.max(train_df[rank_var_list],axis=1)
# train_df['sum'] = np.sum(train_df[rank_var_list],axis=1)
# train_df['median'] = np.median(train_df[rank_var_list],axis=1)

# train_df['rank_of_rank_mean'] = train_df['mean'].rank(method='average',pct=True)
# train_df['rank_of_rank_std'] = train_df['std'].rank(method='average',pct=True)
# train_df['rank_of_rank_min'] = train_df['min'].rank(method='average',pct=True)
# train_df['rank_of_rank_max'] = train_df['max'].rank(method='average',pct=True)
# train_df['rank_of_rank_sum'] = train_df['sum'].rank(method='average',pct=True)
# train_df['rank_of_rank_median'] = train_df['median'].rank(method='average',pct=True)


# #simple stats on rank cols 
# test_df['mean'] = np.mean(test_df[rank_var_list],axis=1)
# test_df['std'] = np.std(test_df[rank_var_list],axis=1)
# test_df['min'] = np.min(test_df[rank_var_list],axis=1)
# test_df['max'] = np.max(test_df[rank_var_list],axis=1)
# test_df['sum'] = np.sum(test_df[rank_var_list],axis=1)
# test_df['median'] = np.median(test_df[rank_var_list],axis=1)

# test_df['rank_of_rank_mean'] = test_df['mean'].rank(method='average',pct=True)
# test_df['rank_of_rank_std'] = test_df['std'].rank(method='average',pct=True)
# test_df['rank_of_rank_min'] = test_df['min'].rank(method='average',pct=True)
# test_df['rank_of_rank_max'] = test_df['max'].rank(method='average',pct=True)
# test_df['rank_of_rank_sum'] = test_df['sum'].rank(method='average',pct=True)
# test_df['rank_of_rank_median'] = test_df['median'].rank(method='average',pct=True)

# train_df[sort_var_list] = train_df[var_list].rank(method='average',pct=True)
# test_df[rank_var_list] = test_df[var_list].rank(method='average',pct=True)


# value_train = train_df[var_list].values
# value_train.sort(axis=1)
# value_train = pd.DataFrame(value_train, train_df.index, sort_var_list)
# train_df[sort_var_list] = value_train

# value_test = test_df[var_list].values
# value_test.sort(axis=1)
# value_test = pd.DataFrame(value_test, test_df.index, sort_var_list)
# test_df[sort_var_list] = value_test

# nlargest = 25
# new_col_names = ['top{}'.format(i) for i in range(1, nlargest+1)]
# order = np.argsort(-train_df[var_list].values, axis=1)[:, :nlargest]
# top_n_train = pd.DataFrame(
#                      #test_df[var_list].columns[order],
#                       order,
#                       columns=new_col_names,
#                       index=train_df.index)

# train_df[new_col_names] = top_n_train

# order = np.argsort(-test_df[var_list].values, axis=1)[:, :nlargest]
# top_n_train = pd.DataFrame(
#                      #test_df[var_list].columns[order],
#                       order,
#                       columns=new_col_names,
#                       index=train_df.index)

# test_df[new_col_names] = top_n_train


# nsmallest = 25
# new_col_names = ['bottom{}'.format(i) for i in range(1, nsmallest+1)]
# order = np.argsort(train_df[var_list].values, axis=1)[:, :nsmallest]
# top_n_train = pd.DataFrame(
#                      #test_df[var_list].columns[order],
#                       order,
#                       columns=new_col_names,
#                       index=train_df.index)

# train_df[new_col_names] = top_n_train
# order = np.argsort(test_df[var_list].values, axis=1)[:, :nsmallest]
# top_n_train = pd.DataFrame(
#                      #test_df[var_list].columns[order],
#                       order,
#                       columns=new_col_names,
#                       index=test_df.index)

# test_df[new_col_names] = top_n_train

In [22]:
test_df.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_rank,var_191_rank,var_192_rank,var_193_rank,var_194_rank,var_195_rank,var_196_rank,var_197_rank,var_198_rank,var_199_rank
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-0.268787,10.906132,-0.009363,1.006862,1.202095,2.359522,2.693048,10.523878,6.872210,-2.781563
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.015128,5.907633,0.246319,9.745025,3.802263,0.316097,-0.408007,8.230675,16.326953,-0.987550
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.152977,9.615162,1.036128,0.843175,0.683044,1.991332,-0.254879,0.173897,18.142648,-0.598898
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,8.735990,6.343545,0.725748,1.843752,3.121276,3.121543,2.345314,5.775775,2.223042,-1.912672
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,2.486604,6.513938,0.437333,1.672594,13.098430,-0.133699,-0.502636,0.299454,3.593674,-2.800706


## Model

In [23]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_folds = 5
random_seed = 26
model = 'rank_multiply_original_cols'


model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: rank_multiply_original_cols_5_folds


In [29]:
#feats = train_df.iloc[:,2:].columns.tolist();feats

#exclusion = ['ID_code', 'target']+ rank_var_list
exclusion = ['ID_code', 'target']

feats = [c for c in train_df.columns if c not in exclusion]

#cat_feats = [c for c in feats if c not in var_list]

In [30]:
feats

['var_0',
 'var_1',
 'var_2',
 'var_3',
 'var_4',
 'var_5',
 'var_6',
 'var_7',
 'var_8',
 'var_9',
 'var_10',
 'var_11',
 'var_12',
 'var_13',
 'var_14',
 'var_15',
 'var_16',
 'var_17',
 'var_18',
 'var_19',
 'var_20',
 'var_21',
 'var_22',
 'var_23',
 'var_24',
 'var_25',
 'var_26',
 'var_27',
 'var_28',
 'var_29',
 'var_30',
 'var_31',
 'var_32',
 'var_33',
 'var_34',
 'var_35',
 'var_36',
 'var_37',
 'var_38',
 'var_39',
 'var_40',
 'var_41',
 'var_42',
 'var_43',
 'var_44',
 'var_45',
 'var_46',
 'var_47',
 'var_48',
 'var_49',
 'var_50',
 'var_51',
 'var_52',
 'var_53',
 'var_54',
 'var_55',
 'var_56',
 'var_57',
 'var_58',
 'var_59',
 'var_60',
 'var_61',
 'var_62',
 'var_63',
 'var_64',
 'var_65',
 'var_66',
 'var_67',
 'var_68',
 'var_69',
 'var_70',
 'var_71',
 'var_72',
 'var_73',
 'var_74',
 'var_75',
 'var_76',
 'var_77',
 'var_78',
 'var_79',
 'var_80',
 'var_81',
 'var_82',
 'var_83',
 'var_84',
 'var_85',
 'var_86',
 'var_87',
 'var_88',
 'var_89',
 'var_90',
 'var_91'

In [ ]:
clfs = []
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
oof_preds = np.zeros((len(train_df), 1))
test_preds = np.zeros((len(test_df), 1))


X = train_df[feats]
y = train_df['target']
X_test = test_df[feats]
test_ids = test_df.ID_code.values


parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    #'scale_pos_weight': 400,
    #'device' : 'gpu' ,
    'boosting': 'gbdt',
    'num_leaves': 31, #31
    'feature_fraction': 0.5,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}

feature_importance_df = pd.DataFrame()
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
    print("Current Fold: {}".format(fold_+1))
    trn_x, trn_y = X.iloc[trn_, :], y[trn_]
    val_x, val_y = X.iloc[val_, :], y[val_]


    trn_lgb = lgb.Dataset(trn_x, trn_y)
    val_lgb = lgb.Dataset(val_x, val_y)
    clf = lgb.train(parameters,
                     train_set=trn_lgb,
                     #valid_sets=[valid_data_lgb,holdout_data_lgb],
                     valid_sets=[trn_lgb, val_lgb],
                     num_boost_round=3000,
                     early_stopping_rounds=50,
                     verbose_eval=False)
    


    val_pred = clf.predict(val_x)
    test_fold_pred = clf.predict(X_test)

    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))
    
    print('getting feature importance')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
test_preds /= n_folds
roc_score = roc_auc_score(y, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))


print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/{}_{}.csv'.format(model_name,str(roc_score)), index=False)

display_importances(feature_importance_df)


Current Fold: 1
AUC = 0.8914939421455298
getting feature importance
Current Fold: 2
AUC = 0.8897632609029004
getting feature importance
Current Fold: 3
AUC = 0.8945215141634795
getting feature importance
Current Fold: 4
AUC = 0.893457093259491
getting feature importance
Current Fold: 5


In [75]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "NOTEBOOK_FULL_PATH = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [76]:


shutil.copyfile(os.path.basename(NOTEBOOK_FULL_PATH), 
                             '../models/{}_{}.ipynb'.format(model_name, str(roc_score)))


'../models/index of top and bottom 25_5_folds_0.8597064708771092.ipynb'